---
title: "大規模な不均衡データに対するロジスティック回帰（前編）"
subtitle: "離散時間 MCMC から連続時間 MCMC へ"
author: "司馬博文"
date: 7/12/2024
date-modified: 10/7/2024
image: Logistic_WhiteBackground.svg
categories: [Bayesian, Computation, Julia, MCMC, Statistics]
bibliography: 
    - ../../../assets/2023.bib
    - ../../../assets/2024.bib
    - ../../../assets/2025.bib
csl: ../../../assets/apalike.csl
abstract-title: 概要
abstract: ロジットモデルやプロビットモデルの事後分布からのサンプリングには，その混合構造を利用したデータ拡張による Gibbs サンプラーが考案されている．しかし，このような Gibbs サンプラーは不明な理由で極めて収束が遅くなることがよく見られ，そのうちの１つのパターンが **大規模な不均衡データ** である．この記事では，この現象がなぜ起こるかに関する考察を与え，次稿で代替手法として Zig-Zag サンプラーがうまくいくことをみる．
code-fold: false
execute:
    cache: true
listing: 
    -   id: lst-listing
        type: grid
        grid-columns: 1
        grid-item-align: center
        sort: false
        contents:
            - "Logistic2.qmd"
        date-format: iso
        fields: [title,image,date,subtitle]
    -   id: lst-listing2
        type: grid
        grid-columns: 1
        grid-item-align: center
        sort: false
        contents:
            - "../TransDimensionalModels/IdealPoint.qmd"
        date-format: iso
        fields: [title,image,date,subtitle]
    -   id: lst-listing3
        type: grid
        grid-columns: 1
        grid-item-align: center
        sort: false
        contents:
            - "../Julia/MALAwithJulia.qmd"
        date-format: iso
        fields: [title,image,date,subtitle]
---

::: {.hidden}

::: {.content-visible when-format="html"}

A Blog Entry on Bayesian Computation by an Applied Mathematician

$$

\renewcommand{\P}{\operatorname{P}}\newcommand{\E}{\operatorname{E}}
\newcommand{\R}{\mathbb{R}}\newcommand{\F}{\mathcal{F}}
\newcommand{\abs}[1]{\lvert#1\rvert}\newcommand{\Abs}[1]{\left|#1\right|}\newcommand{\ABs}[1]{\biggl|#1\biggr|}\newcommand{\norm}[1]{\|#1\|}\newcommand{\Norm}[1]{\left\|#1\right\|}\newcommand{\NOrm}[1]{\biggl\|#1\biggr\|}\newcommand{\Brace}[1]{\left\{#1\right\}}\newcommand{\BRace}[1]{\biggl\{#1\biggr\}}\newcommand{\paren}[1]{\left(#1\right)}\newcommand{\Paren}[1]{\biggr(#1\biggl)}\newcommand{\brac}[1]{\langle#1\rangle}\newcommand{\Brac}[1]{\left\langle#1\right\rangle}\newcommand{\BRac}[1]{\biggl\langle#1\biggr\rangle}\newcommand{\bra}[1]{\left\langle#1\right|}\newcommand{\ket}[1]{\left|#1\right\rangle}\newcommand{\Square}[1]{\left[#1\right]}\newcommand{\SQuare}[1]{\biggl[#1\biggr]}\newcommand{\rN}{\operatorname{N}}\newcommand{\ov}[1]{\overline{#1}}\newcommand{\un}[1]{\underline{#1}}\newcommand{\wt}[1]{\widetilde{#1}}\newcommand{\wh}[1]{\widehat{#1}}\newcommand{\pp}[2]{\frac{\partial #1}{\partial #2}}\newcommand{\ppp}[3]{\frac{\partial #1}{\partial #2\partial #3}}\newcommand{\dd}[2]{\frac{d #1}{d #2}}\newcommand{\floor}[1]{\lfloor#1\rfloor}\newcommand{\Floor}[1]{\left\lfloor#1\right\rfloor}\newcommand{\ceil}[1]{\lceil#1\rceil}\newcommand{\ocinterval}[1]{(#1]}\newcommand{\cointerval}[1]{[#1)}\newcommand{\COinterval}[1]{\left[#1\right)}\newcommand{\iso}{\overset{\sim}{\to}}



\newcommand{\y}{\b{y}}\newcommand{\mi}{\,|\,}\newcommand{\Mark}{\mathrm{Mark}}
\newcommand{\argmax}{\operatorname*{argmax}}\newcommand{\argmin}{\operatorname*{argmin}}

\newcommand{\pr}{\mathrm{pr}}\newcommand{\Conv}{\operatorname{Conv}}\newcommand{\cU}{\mathcal{U}}
\newcommand{\Map}{\mathrm{Map}}\newcommand{\dom}{\mathrm{Dom}\;}\newcommand{\cod}{\mathrm{Cod}\;}\newcommand{\supp}{\mathrm{supp}\;}
\newcommand{\grad}{\operatorname{grad}}\newcommand{\rot}{\operatorname{rot}}\renewcommand{\div}{\operatorname{div}}\newcommand{\tr}{\operatorname{tr}}\newcommand{\Tr}{\operatorname{Tr}}\newcommand{\KL}{\operatorname{KL}}\newcommand{\JS}{\operatorname{JS}}\newcommand{\ESS}{\operatorname{ESS}}\newcommand{\MSE}{\operatorname{MSE}}\newcommand{\erf}{\operatorname{erf}}\newcommand{\arctanh}{\operatorname{arctanh}}\newcommand{\pl}{\operatorname{pl}}\newcommand{\minimize}{\operatorname{minimize}}\newcommand{\subjectto}{\operatorname{subject to}}\newcommand{\sinc}{\operatorname{sinc}}\newcommand{\Ent}{\operatorname{Ent}}\newcommand{\Polya}{\operatorname{Polya}}\newcommand{\Exp}{\operatorname{Exp}}\newcommand{\codim}{\operatorname{codim}}\newcommand{\sgn}{\operatorname{sgn}}\newcommand{\rank}{\operatorname{rank}}

\newcommand{\vctr}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\vctrr}[3]{\begin{pmatrix}#1\\#2\\#3\end{pmatrix}}\newcommand{\mtrx}[4]{\begin{pmatrix}#1&#2\\#3&#4\end{pmatrix}}\newcommand{\smtrx}[4]{\paren{\begin{smallmatrix}#1&#2\\#3&#4\end{smallmatrix}}}\newcommand{\Ker}{\mathrm{Ker}\;}\newcommand{\Coker}{\mathrm{Coker}\;}\newcommand{\Coim}{\mathrm{Coim}\;}\newcommand{\lcm}{\mathrm{lcm}}\newcommand{\GL}{\mathrm{GL}}\newcommand{\SL}{\mathrm{SL}}\newcommand{\alt}{\mathrm{alt}}

\renewcommand{\Re}{\mathrm{Re}\;}\renewcommand{\Im}{\mathrm{Im}\,}\newcommand{\Gal}{\mathrm{Gal}}\newcommand{\PGL}{\mathrm{PGL}}\newcommand{\PSL}{\mathrm{PSL}}\newcommand{\Log}{\mathrm{Log}\,}\newcommand{\Res}{\mathrm{Res}\,}\newcommand{\on}{\mathrm{on}\;}\newcommand{\hatC}{\widehat{\C}}\newcommand{\hatR}{\hat{\R}}\newcommand{\PV}{\mathrm{P.V.}}\newcommand{\diam}{\mathrm{diam}}\newcommand{\Area}{\mathrm{Area}}\newcommand{\Lap}{\Laplace}\newcommand{\f}{\mathbf{f}}\newcommand{\cR}{\mathcal{R}}\newcommand{\const}{\mathrm{const.}}\newcommand{\Om}{\Omega}\newcommand{\Cinf}{C^\infty}\newcommand{\ep}{\epsilon}\newcommand{\dist}{\mathrm{dist}}\newcommand{\opart}{\o{\partial}}\newcommand{\Length}{\mathrm{Length}}

\newcommand{\cA}{\mathcal{A}}\newcommand{\cO}{\mathcal{O}}\newcommand{\cW}{\mathcal{W}}\renewcommand{\O}{\mathcal{O}}\renewcommand{\S}{\mathcal{S}}\newcommand{\U}{\mathcal{U}}\newcommand{\V}{\mathrm{V}}\newcommand{\N}{\mathbb{N}}\newcommand{\bN}{\mathbb{N}}\newcommand{\C}{\mathrm{C}}\newcommand{\bC}{\mathbb{C}}\newcommand{\Z}{\mathcal{Z}}\newcommand{\Q}{\mathbb{Q}}\newcommand{\bQ}{\mathbb{Q}}\newcommand{\TV}{\mathrm{TV}}\newcommand{\ORD}{\mathrm{ORD}}\newcommand{\Card}{\mathrm{Card}\,}\newcommand{\Top}{\mathrm{Top}}\newcommand{\Disc}{\mathrm{Disc}}\newcommand{\Codisc}{\mathrm{Codisc}}\newcommand{\CoDisc}{\mathrm{CoDisc}}\newcommand{\Ult}{\mathrm{Ult}}\newcommand{\ord}{\mathrm{ord}}\newcommand{\bS}{\mathbb{S}}\newcommand{\PConn}{\mathrm{PConn}}\newcommand{\mult}{\mathrm{mult}}\newcommand{\inv}{\mathrm{inv}}

\newcommand{\Der}{\mathrm{Der}}\newcommand{\osub}{\overset{\mathrm{open}}{\subset}}\newcommand{\osup}{\overset{\mathrm{open}}{\supset}}\newcommand{\al}{\alpha}\newcommand{\K}{\mathbb{K}}\newcommand{\Sp}{\mathrm{Sp}}\newcommand{\g}{\mathfrak{g}}\newcommand{\h}{\mathfrak{h}}\newcommand{\Imm}{\mathrm{Imm}}\newcommand{\Imb}{\mathrm{Imb}}\newcommand{\Gr}{\mathrm{Gr}}

\newcommand{\Ad}{\mathrm{Ad}}\newcommand{\finsupp}{\mathrm{fin\;supp}}\newcommand{\SO}{\mathrm{SO}}\newcommand{\SU}{\mathrm{SU}}\newcommand{\acts}{\curvearrowright}\newcommand{\mono}{\hookrightarrow}\newcommand{\epi}{\twoheadrightarrow}\newcommand{\Stab}{\mathrm{Stab}}\newcommand{\nor}{\mathrm{nor}}\newcommand{\T}{\mathbb{T}}\newcommand{\Aff}{\mathrm{Aff}}\newcommand{\rsup}{\triangleright}\newcommand{\subgrp}{\overset{\mathrm{subgrp}}{\subset}}\newcommand{\Ext}{\mathrm{Ext}}\newcommand{\sbs}{\subset}\newcommand{\sps}{\supset}\newcommand{\In}{\mathrm{in}\;}\newcommand{\Tor}{\mathrm{Tor}}\newcommand{\p}{\b{p}}\newcommand{\q}{\mathfrak{q}}\newcommand{\m}{\mathfrak{m}}\newcommand{\cS}{\mathcal{S}}\newcommand{\Frac}{\mathrm{Frac}\,}\newcommand{\Spec}{\mathrm{Spec}\,}\newcommand{\bA}{\mathbb{A}}\newcommand{\Sym}{\mathrm{Sym}}\newcommand{\Ann}{\mathrm{Ann}}\newcommand{\Her}{\mathrm{Her}}\newcommand{\Bil}{\mathrm{Bil}}\newcommand{\Ses}{\mathrm{Ses}}\newcommand{\FVS}{\mathrm{FVS}}

\newcommand{\Ho}{\mathrm{Ho}}\newcommand{\CW}{\mathrm{CW}}\newcommand{\lc}{\mathrm{lc}}\newcommand{\cg}{\mathrm{cg}}\newcommand{\Fib}{\mathrm{Fib}}\newcommand{\Cyl}{\mathrm{Cyl}}\newcommand{\Ch}{\mathrm{Ch}}
\newcommand{\rP}{\mathrm{P}}\newcommand{\rE}{\mathrm{E}}\newcommand{\e}{\b{e}}\renewcommand{\k}{\b{k}}\newcommand{\Christ}[2]{\begin{Bmatrix}#1\\#2\end{Bmatrix}}\renewcommand{\Vec}[1]{\overrightarrow{\mathrm{#1}}}\newcommand{\hen}[1]{\mathrm{#1}}\renewcommand{\b}[1]{\boldsymbol{#1}}

\newcommand{\Inc}{\mathrm{Inc}}\newcommand{\aInc}{\mathrm{aInc}}\newcommand{\HS}{\mathrm{HS}}\newcommand{\loc}{\mathrm{loc}}\newcommand{\Lh}{\mathrm{L.h.}}\newcommand{\Epi}{\mathrm{Epi}}\newcommand{\slim}{\mathrm{slim}}\newcommand{\Ban}{\mathrm{Ban}}\newcommand{\Hilb}{\mathrm{Hilb}}\newcommand{\Ex}{\mathrm{Ex}}\newcommand{\Co}{\mathrm{Co}}\newcommand{\sa}{\mathrm{sa}}\newcommand{\nnorm}[1]{{\left\vert\kern-0.25ex\left\vert\kern-0.25ex\left\vert #1 \right\vert\kern-0.25ex\right\vert\kern-0.25ex\right\vert}}\newcommand{\dvol}{\mathrm{dvol}}\newcommand{\Sconv}{\mathrm{Sconv}}\newcommand{\I}{\mathcal{I}}\newcommand{\nonunital}{\mathrm{nu}}\newcommand{\cpt}{\mathrm{cpt}}\newcommand{\lcpt}{\mathrm{lcpt}}\newcommand{\com}{\mathrm{com}}\newcommand{\Haus}{\mathrm{Haus}}\newcommand{\proper}{\mathrm{proper}}\newcommand{\infinity}{\mathrm{inf}}\newcommand{\TVS}{\mathrm{TVS}}\newcommand{\ess}{\mathrm{ess}}\newcommand{\ext}{\mathrm{ext}}\newcommand{\Index}{\mathrm{Index}\;}\newcommand{\SSR}{\mathrm{SSR}}\newcommand{\vs}{\mathrm{vs.}}\newcommand{\fM}{\mathfrak{M}}\newcommand{\EDM}{\mathrm{EDM}}\newcommand{\Tw}{\mathrm{Tw}}\newcommand{\fC}{\mathfrak{C}}\newcommand{\bn}{\boldsymbol{n}}\newcommand{\br}{\boldsymbol{r}}\newcommand{\Lam}{\Lambda}\newcommand{\lam}{\lambda}\newcommand{\one}{\mathbf{1}}\newcommand{\dae}{\text{-a.e.}}\newcommand{\das}{\text{-a.s.}}\newcommand{\td}{\text{-}}\newcommand{\RM}{\mathrm{RM}}\newcommand{\BV}{\mathrm{BV}}\newcommand{\normal}{\mathrm{normal}}\newcommand{\lub}{\mathrm{lub}\;}\newcommand{\Graph}{\mathrm{Graph}}\newcommand{\Ascent}{\mathrm{Ascent}}\newcommand{\Descent}{\mathrm{Descent}}\newcommand{\BIL}{\mathrm{BIL}}\newcommand{\fL}{\mathfrak{L}}\newcommand{\De}{\Delta}

\newcommand{\calA}{\mathcal{A}}\newcommand{\calB}{\mathcal{B}}\newcommand{\D}{\mathcal{D}}\newcommand{\Y}{\mathcal{Y}}\newcommand{\calC}{\mathcal{C}}\renewcommand{\ae}{\mathrm{a.e.}\;}\newcommand{\cZ}{\mathcal{Z}}\newcommand{\fF}{\mathfrak{F}}\newcommand{\fI}{\mathfrak{I}}\newcommand{\rV}{\mathrm{V}}\newcommand{\cE}{\mathcal{E}}\newcommand{\sMap}{\sigma\textrm{-}\mathrm{Map}}\newcommand{\cC}{\mathcal{C}}\newcommand{\comp}{\complement}\newcommand{\J}{\mathcal{J}}\newcommand{\sumN}[1]{\sum_{#1\in\N}}\newcommand{\cupN}[1]{\cup_{#1\in\N}}\newcommand{\capN}[1]{\cap_{#1\in\N}}\newcommand{\Sum}[1]{\sum_{#1=1}^\infty}\newcommand{\sumn}{\sum_{n=1}^\infty}\newcommand{\summ}{\sum_{m=1}^\infty}\newcommand{\sumk}{\sum_{k=1}^\infty}\newcommand{\sumi}{\sum_{i=1}^\infty}\newcommand{\sumj}{\sum_{j=1}^\infty}\newcommand{\cupn}{\cup_{n=1}^\infty}\newcommand{\capn}{\cap_{n=1}^\infty}\newcommand{\cupk}{\cup_{k=1}^\infty}\newcommand{\cupi}{\cup_{i=1}^\infty}\newcommand{\cupj}{\cup_{j=1}^\infty}\newcommand{\limn}{\lim_{n\to\infty}}\renewcommand{\L}{\mathcal{L}}\newcommand{\cL}{\mathcal{L}}\newcommand{\Cl}{\mathrm{Cl}}\newcommand{\cN}{\mathcal{N}}\newcommand{\Ae}{\textrm{-a.e.}\;}\renewcommand{\csub}{\overset{\textrm{closed}}{\subset}}\renewcommand{\csup}{\overset{\textrm{closed}}{\supset}}\newcommand{\wB}{\wt{B}}\newcommand{\cG}{\mathcal{G}}\newcommand{\Lip}{\mathrm{Lip}}\newcommand{\AC}{\mathrm{AC}}\newcommand{\Mol}{\mathrm{Mol}}

\newcommand{\Pe}{\mathrm{Pe}}\newcommand{\wR}{\wh{\mathbb{\R}}}\newcommand*{\Laplace}{\mathop{}\!\mathbin\bigtriangleup}\newcommand*{\DAlambert}{\mathop{}\!\mathbin\Box}\newcommand{\bT}{\mathbb{T}}\newcommand{\dx}{\dslash x}\newcommand{\dt}{\dslash t}\newcommand{\ds}{\dslash s}

\newcommand{\round}{\mathrm{round}}\newcommand{\cond}{\mathrm{cond}}\newcommand{\diag}{\mathrm{diag}}
\newcommand{\Adj}{\mathrm{Adj}}\newcommand{\Pf}{\mathrm{Pf}}\newcommand{\Sg}{\mathrm{Sg}}


\newcommand{\aseq}{\overset{\text{a.s.}}{=}}\newcommand{\deq}{\overset{\text{d}}{=}}\newcommand{\cV}{\mathcal{V}}\newcommand{\FM}{\mathrm{FM}}\newcommand{\KR}{\mathrm{KR}}\newcommand{\rba}{\mathrm{rba}}\newcommand{\rca}{\mathrm{rca}}\newcommand{\Prob}{\mathrm{Prob}}\newcommand{\X}{\mathcal{X}}\newcommand{\Meas}{\mathrm{Meas}}\newcommand{\as}{\;\text{a.s.}}\newcommand{\io}{\;\mathrm{i.o.}}\newcommand{\fe}{\;\text{f.e.}}\newcommand{\bF}{\mathbb{F}}\newcommand{\W}{\mathcal{W}}\newcommand{\Pois}{\mathrm{Pois}}\newcommand{\iid}{\text{i.i.d.}}\newcommand{\wconv}{\rightsquigarrow}\newcommand{\Var}{\mathrm{Var}}\newcommand{\xrightarrown}{\xrightarrow{n\to\infty}}\newcommand{\au}{\mathrm{au}}\newcommand{\cT}{\mathcal{T}}\newcommand{\wto}{\overset{\text{w}}{\to}}\newcommand{\dto}{\overset{\text{d}}{\to}}\newcommand{\sto}{\overset{\text{s}}{\to}}\newcommand{\pto}{\overset{\text{p}}{\to}}\newcommand{\mto}{\overset{\text{m}}{\to}}\newcommand{\vto}{\overset{v}{\to}}\newcommand{\Cont}{\mathrm{Cont}}\newcommand{\stably}{\mathrm{stably}}\newcommand{\Np}{\mathbb{N}^+}\newcommand{\oM}{\overline{\mathcal{M}}}\newcommand{\fP}{\mathfrak{P}}\newcommand{\sign}{\mathrm{sign}}
\newcommand{\Borel}{\mathrm{Borel}}\newcommand{\Mid}{\,|\,}\newcommand{\middleMid}{\;\middle|\;}\newcommand{\CP}{\mathrm{CP}}\newcommand{\bD}{\mathbb{D}}\newcommand{\bL}{\mathbb{L}}\newcommand{\fW}{\mathfrak{W}}\newcommand{\DL}{\mathcal{D}\mathcal{L}}\renewcommand{\r}[1]{\mathrm{#1}}\newcommand{\rC}{\mathrm{C}}\newcommand{\qqquad}{\qquad\quad}

\newcommand{\bit}{\mathrm{bit}}

\newcommand{\err}{\mathrm{err}}

\newcommand{\varparallel}{\mathbin{\!/\mkern-5mu/\!}}\newcommand{\Ri}{\mathrm{Ri}}\newcommand{\Cone}{\mathrm{Cone}}\newcommand{\Int}{\mathrm{Int}}

\newcommand{\pre}{\mathrm{pre}}\newcommand{\om}{\omega}


\newcommand{\del}{\partial}
\newcommand{\LHS}{\mathrm{LHS}}\newcommand{\RHS}{\mathrm{RHS}}\newcommand{\bnu}{\boldsymbol{\nu}}\newcommand{\interior}{\mathrm{in}\;}\newcommand{\SH}{\mathrm{SH}}\renewcommand{\v}{\boldsymbol{\nu}}\newcommand{\n}{\mathbf{n}}\newcommand{\ssub}{\Subset}\newcommand{\curl}{\mathrm{curl}}

\newcommand{\Ei}{\mathrm{Ei}}\newcommand{\sn}{\mathrm{sn}}\newcommand{\wgamma}{\widetilde{\gamma}}

\newcommand{\Ens}{\mathrm{Ens}}

\newcommand{\cl}{\mathrm{cl}}\newcommand{\x}{\boldsymbol{x}}

\newcommand{\Do}{\mathrm{Do}}\newcommand{\IV}{\mathrm{IV}}

\newcommand{\AIC}{\mathrm{AIC}}\newcommand{\mrl}{\mathrm{mrl}}\newcommand{\dotx}{\dot{x}}\newcommand{\UMV}{\mathrm{UMV}}\newcommand{\BLU}{\mathrm{BLU}}

\newcommand{\comb}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\bP}{\mathbb{P}}\newcommand{\compsub}{\overset{\textrm{cpt}}{\subset}}\newcommand{\lip}{\textrm{lip}}\newcommand{\BL}{\mathrm{BL}}\newcommand{\G}{\mathbb{G}}\newcommand{\NB}{\mathrm{NB}}\newcommand{\oR}{\ov{\R}}\newcommand{\liminfn}{\liminf_{n\to\infty}}\newcommand{\limsupn}{\limsup_{n\to\infty}}\newcommand{\esssup}{\mathrm{ess.sup}}\newcommand{\asto}{\xrightarrow{\as}}\newcommand{\Cov}{\mathrm{Cov}}\newcommand{\cQ}{\mathcal{Q}}\newcommand{\VC}{\mathrm{VC}}\newcommand{\mb}{\mathrm{mb}}\newcommand{\Avar}{\mathrm{Avar}}\newcommand{\bB}{\mathbb{B}}\newcommand{\bW}{\mathbb{W}}\newcommand{\sd}{\mathrm{sd}}\newcommand{\w}[1]{\widehat{#1}}\newcommand{\bZ}{\mathbb{Z}}\newcommand{\Bernoulli}{\mathrm{Ber}}\newcommand{\Ber}{\mathrm{Ber}}\newcommand{\Mult}{\mathrm{Mult}}\newcommand{\BPois}{\mathrm{BPois}}\newcommand{\fraks}{\mathfrak{s}}\newcommand{\frakk}{\mathfrak{k}}\newcommand{\IF}{\mathrm{IF}}\newcommand{\bX}{\boldsymbol{X}}\newcommand{\bx}{\boldsymbol{x}}\newcommand{\indep}{\perp\!\!\!\perp}\newcommand{\IG}{\mathrm{IG}}\newcommand{\Levy}{\mathrm{Levy}}\newcommand{\MP}{\mathrm{MP}}\newcommand{\Hermite}{\mathrm{Hermite}}\newcommand{\Skellam}{\mathrm{Skellam}}\newcommand{\Dirichlet}{\mathrm{Dirichlet}}\renewcommand{\Beta}{\operatorname{Beta}}\newcommand{\bE}{\mathbb{E}}\newcommand{\bG}{\mathbb{G}}\newcommand{\MISE}{\mathrm{MISE}}\newcommand{\logit}{\mathtt{logit}}\newcommand{\expit}{\mathtt{expit}}\newcommand{\cK}{\mathcal{K}}\newcommand{\dl}{\dot{l}}\newcommand{\dotp}{\dot{p}}\newcommand{\wl}{\wt{l}}\newcommand{\Gauss}{\mathrm{Gauss}}\newcommand{\fA}{\mathfrak{A}}\newcommand{\under}{\mathrm{under}\;}\newcommand{\whtheta}{\wh{\theta}}\newcommand{\Em}{\mathrm{Em}}\newcommand{\ztheta}{{\theta_0}}
\newcommand{\rO}{\mathrm{O}}\newcommand{\Bin}{\mathrm{Bin}}\newcommand{\rW}{\mathrm{W}}\newcommand{\rG}{\mathrm{G}}\newcommand{\rB}{\mathrm{B}}\newcommand{\rU}{\mathrm{U}}\newcommand{\HG}{\mathrm{HG}}\newcommand{\GAMMA}{\mathrm{Gamma}}\newcommand{\Cauchy}{\mathrm{Cauchy}}\newcommand{\rt}{\mathrm{t}}\newcommand{\rF}{\mathrm{F}}
\newcommand{\FE}{\mathrm{FE}}\newcommand{\bV}{\boldsymbol{V}}\newcommand{\GLS}{\mathrm{GLS}}\newcommand{\be}{\boldsymbol{e}}\newcommand{\POOL}{\mathrm{POOL}}\newcommand{\GMM}{\mathrm{GMM}}\newcommand{\MM}{\mathrm{MM}}\newcommand{\SSIV}{\mathrm{SSIV}}\newcommand{\JIV}{\mathrm{JIV}}\newcommand{\AR}{\mathrm{AR}}\newcommand{\ILS}{\mathrm{ILS}}\newcommand{\SLS}{\mathrm{SLS}}\newcommand{\LIML}{\mathrm{LIML}}

\newcommand{\Rad}{\mathrm{Rad}}\newcommand{\bY}{\boldsymbol{Y}}\newcommand{\pone}{{(1)}}\newcommand{\ptwo}{{(2)}}\newcommand{\ps}[1]{{(#1)}}\newcommand{\fsub}{\overset{\text{finite}}{\subset}}


\newcommand{\varlim}{\varprojlim}\newcommand{\Hom}{\mathrm{Hom}}\newcommand{\Iso}{\mathrm{Iso}}\newcommand{\Mor}{\mathrm{Mor}}\newcommand{\Isom}{\mathrm{Isom}}\newcommand{\Aut}{\mathrm{Aut}}\newcommand{\End}{\mathrm{End}}\newcommand{\op}{\mathrm{op}}\newcommand{\ev}{\mathrm{ev}}\newcommand{\Ob}{\mathrm{Ob}}\newcommand{\Ar}{\mathrm{Ar}}\newcommand{\Arr}{\mathrm{Arr}}\newcommand{\Set}{\mathrm{Set}}\newcommand{\Grp}{\mathrm{Grp}}\newcommand{\Cat}{\mathrm{Cat}}\newcommand{\Mon}{\mathrm{Mon}}\newcommand{\Ring}{\mathrm{Ring}}\newcommand{\CRing}{\mathrm{CRing}}\newcommand{\Ab}{\mathrm{Ab}}\newcommand{\Pos}{\mathrm{Pos}}\newcommand{\Vect}{\mathrm{Vect}}\newcommand{\FinVect}{\mathrm{FinVect}}\newcommand{\FinSet}{\mathrm{FinSet}}\newcommand{\FinMeas}{\mathrm{FinMeas}}\newcommand{\OmegaAlg}{\Omega\text{-}\mathrm{Alg}}\newcommand{\OmegaEAlg}{(\Omega,E)\text{-}\mathrm{Alg}}\newcommand{\Fun}{\mathrm{Fun}}\newcommand{\Func}{\mathrm{Func}}

\newcommand{\Stoch}{\mathrm{Stoch}}\newcommand{\FinStoch}{\mathrm{FinStoch}}\newcommand{\Copy}{\mathrm{copy}}\newcommand{\Delete}{\mathrm{delete}}
\newcommand{\Bool}{\mathrm{Bool}}\newcommand{\CABool}{\mathrm{CABool}}\newcommand{\CompBoolAlg}{\mathrm{CompBoolAlg}}\newcommand{\BoolAlg}{\mathrm{BoolAlg}}\newcommand{\BoolRng}{\mathrm{BoolRng}}\newcommand{\HeytAlg}{\mathrm{HeytAlg}}\newcommand{\CompHeytAlg}{\mathrm{CompHeytAlg}}\newcommand{\Lat}{\mathrm{Lat}}\newcommand{\CompLat}{\mathrm{CompLat}}\newcommand{\SemiLat}{\mathrm{SemiLat}}\newcommand{\Stone}{\mathrm{Stone}}\newcommand{\Mfd}{\mathrm{Mfd}}\newcommand{\LieAlg}{\mathrm{LieAlg}}
\newcommand{\Op}{\mathrm{Op}}
\newcommand{\Sh}{\mathrm{Sh}}
\newcommand{\Diff}{\mathrm{Diff}}
\newcommand{\B}{\mathcal{B}}\newcommand{\cB}{\mathcal{B}}\newcommand{\Span}{\mathrm{Span}}\newcommand{\Corr}{\mathrm{Corr}}\newcommand{\Decat}{\mathrm{Decat}}\newcommand{\Rep}{\mathrm{Rep}}\newcommand{\Grpd}{\mathrm{Grpd}}\newcommand{\sSet}{\mathrm{sSet}}\newcommand{\Mod}{\mathrm{Mod}}\newcommand{\SmoothMnf}{\mathrm{SmoothMnf}}\newcommand{\coker}{\mathrm{coker}}\newcommand{\Ord}{\mathrm{Ord}}\newcommand{\eq}{\mathrm{eq}}\newcommand{\coeq}{\mathrm{coeq}}\newcommand{\act}{\mathrm{act}}

\newcommand{\apf}{\mathrm{apf}}\newcommand{\opt}{\mathrm{opt}}\newcommand{\IS}{\mathrm{IS}}\newcommand{\IR}{\mathrm{IR}}\newcommand{\iidsim}{\overset{\text{i.i.d.}}{\sim}}\newcommand{\propt}{\,\propto\,}\newcommand{\bM}{\mathbb{M}}\newcommand{\cX}{\mathcal{X}}\newcommand{\cY}{\mathcal{Y}}\newcommand{\cP}{\mathcal{P}}\newcommand{\ola}[1]{\overleftarrow{#1}}

\renewcommand{\iff}{\;\mathrm{iff}\;}
\newcommand{\False}{\mathrm{False}}\newcommand{\True}{\mathrm{True}}
\newcommand{\otherwise}{\mathrm{otherwise}}
\newcommand{\suchthat}{\;\mathrm{s.t.}\;}

\newcommand{\cM}{\mathcal{M}}\newcommand{\M}{\mathbb{M}}\newcommand{\cF}{\mathcal{F}}\newcommand{\cD}{\mathcal{D}}\newcommand{\fX}{\mathfrak{X}}\newcommand{\fY}{\mathfrak{Y}}\newcommand{\fZ}{\mathfrak{Z}}\renewcommand{\H}{\mathcal{H}}\newcommand{\cH}{\mathcal{H}}\newcommand{\fH}{\mathfrak{H}}\newcommand{\bH}{\mathbb{H}}\newcommand{\id}{\mathrm{id}}\newcommand{\A}{\mathcal{A}}
\newcommand{\lmd}{\lambda}
\newcommand{\Lmd}{\Lambda}
\newcommand{\cI}{\mathcal{I}}

\newcommand{\Lrarrow}{\;\;\Leftrightarrow\;\;}
\DeclareMathOperator{\des}{des}
\DeclareMathOperator{\nd}{nd}
\DeclareMathOperator{\dsep}{d-sep}
\DeclareMathOperator{\sep}{sep}
\newcommand{\rLL}{\mathrm{LL}}\newcommand{\HT}{\mathrm{HT}}\newcommand{\PS}{\mathrm{PS}}\newcommand{\rI}{\mathrm{I}}
$$

:::

:::



## 二項回帰モデル

### はじめに

独立標本 $[n]:=\{1,\cdots,n\}$ 上に，２値データ $y_i\in\{0,1\}=2$ が観測されているとする．$y_i$ は例えば症状の有無などのように，２つのカテゴリーのみを持つデータである．

$x_i\in\R^p$ を $p$ 次元の実ベクトルとし，これを説明変数として $y_i$ をよく予測するモデルを作りたいとする．

予測確率を最大限上げることよりも，$x_i$ の $p$ 個の成分のうちどの成分が $y_i$ に影響を与えているかを知りたい場合，機械学習的な方法よりも，モデルを立てて推定する統計的な方法が適している [@Breiman2001DM]

よく使われる統計モデルは，$y_i=1$ となる確率を予測するという **二項回帰モデル** (binary regression model) である：
$$
\P[Y=1|X=x]=F(\xi^\top x).
$$ {#eq-br-model}

ただし，$F$ は分布関数とする．

これは $Y\sim\Ber(\mu)$ というモデルの平均パラメータ $\mu$ に対して，リンク関数 $G:=F^{-1}$ を通じて線型予測をする **一般化線型モデル** の一種である．

::: {.callout-tip appearance="simple" icon="false" title="二項モデルの尤度"}

尤度は次のように表せる：
$$
p(\{x_i,y_i\}|\xi)=\prod_{i=1}^n\Paren{y_iF(\xi^\top x_i)+(1-y_i)\paren{1-F(\xi^\top x_i)}}.
$$

仮に $y_i\in\{\pm1\}$ であった場合は，次のような簡単な表記もできる：
$$
p(\{x_i,y_i\}|\xi)=\prod_{i=1}^nF(y_i\xi^\top x_i).
$$

:::

### 潜在変数モデルとしての解釈

二項回帰模型 ([-@eq-br-model]) は，$F$ を分布関数にもつ誤差項 $\ep$ を通じて，
$$
Y=1_{\Brace{Y^*>0}},\qquad Y^*=\xi^\top X+\ep,\;\ep\sim F,
$$ {#eq-br-model-latent}
によって $Y$ が観測されていると解釈することもできる．

この解釈は計量経済学において **二項選択モデル** とも呼ばれる [@Hansen2022 p.804]．

この $Y^*$ という潜在変数を考慮することはベイズ計算上も大変有意義である．

$(\xi,Y^*)$ 上の拡張されたパラメータ空間上で事後分布推定をし，その後 $\xi$ 上の周辺分布を求める方法は **データ拡張** と呼ばれる．

問題の次元を上げているため一見非効率に思えるが，式 ([-@eq-br-model-latent]) が人間にとって読みやすいように，その分条件付き確率の構造が単純になっており，Gibbs サンプラーの構成が可能になる．

### ロジットモデル

$F$ を標準ロジスティック分布関数
$$
F(x)=\frac{1}{1+e^{-x}},\qquad G(x)=F^{-1}(x)=\log\frac{x}{1-x},
$$
と取った場合，$G$ を [ロジット](https://ja.wikipedia.org/wiki/ロジット)，このモデルは **ロジットモデル** と呼ばれる：
$$
\P[Y=1\mi X=x]=F(\xi^\top x)=\frac{\exp(\xi^\top x)}{1+\exp(\xi^\top x)}.
$$ {#eq-the-model}

ロジット $G(\mu)$ は $\mu$ の [**対数オッズ**](https://ja.wikipedia.org/wiki/オッズ) ともいい，ロジットモデルは対数オッズに線型モデルをおいているとも解釈される．

このとき，$e^0=1$ であることを利用すれば尤度は次のように表示できる：
$$
p(\{x_i,y_i\}|\xi)=\prod_{i=1}^n\frac{\exp(y_i\xi^\top x_i)}{1+\exp(\xi^\top x_i)}.
$$

::: {.callout-caution title="[@Rasch1960] のモデル" icon="false"}

$$
\P[Y=1\mi X=x,B=b,A=a]=F(ax-b)=\Paren{1+e^{b-ax}}^{-1}
$$
とした場合を **２母数応答モデル** (2PLM: two-parameter logistic model) という．

このようなモデルは項目反応理論で考えられる．

ロジットモデルとの違いは，$x$ も所与ではなく推定するパラメータである点である．$x$ を能力パラメータ，$a$ を項目識別パラメータ，$b$ を難易度パラメータなどという．詳しくは次稿参照：

::: {#lst-listing2}
:::

:::

## ロジットモデルのベイズ推定法

### はじめに

ロジットモデルは比較的単純であるが，多くの深い性質を持っている．

まず $p\ll n$ の仮定が満たされない場合，最尤推定量はバイアスを持つ [@Sur-Candes2019]．

比例的高次元極限では従来の漸近論は成り立たず，尤度比検定も実行不可能になる．

このように $p$ の数が大きい場合などはベイズ推論が志向される理由がある [@Firth1993], [@Gelman+2008]．

### ロジットモデルの事後分布

ロジットモデルではパラメータ $\xi$ をベイズ推定することを考える．

即ち，データ $\{(y_i,x_i)\}_{i=1}^n\subset2\times\R^p$ と事前分布 $p(\xi)d\xi\in\cP(\R^p)$ を通じて，事後分布
\begin{align*}
  p(\xi|\{y_i,x_i\})&\propt p(\xi)p(\{y_i,x_i\}_{i=1}^n|\xi)\\
  &= p(\xi)\prod_{i=1}^n\frac{\exp(y_i(x_i)^\top\xi)}{1+\exp((x_i)^\top\xi)}=:e^{-U(\xi)}
\end{align*}
を計算することを考える．

事後分布は正規化定数を除いて
$$
p(\xi|\{y_i,x_i\})\propt e^{-U(\xi)}
$$
と得られたことになる．

$U$ は **ポテンシャル** と呼び，次のように表せる：
\begin{align*}
    U(\xi)&:=-\log p_0(\xi)-\sum_{i=1}^n\log\paren{\frac{\exp(y_i(x_i)^\top\xi)}{1+\exp((x_i)^\top\xi)}}\\
    &=:U_0(\xi)+U_1(\xi).
\end{align*}

### ロジットモデルの事後分布サンプラー

ロジットリンク $G$ による変換が複雑であるため，ロジスティック回帰は（完全な）ベイズ推定を実行することが難しいモデルとして知られてきた．

一方でリンク関数 $g$ を標準正規分布 $\rN(0,1)$ の分布関数 $\Phi$ の逆関数に取り替えた [プロビットモデル](https://ja.wikipedia.org/wiki/プロビット) にはデータ拡張に基づく Gibbs サンプラーが早くから提案されており [@Albert-Chib1993]，これにより効率的なベイズ推論が可能となっていた．

プロビットモデルはロジットモデルに極めて似ており，ただ裾の重さが違うのみであると言って良い [@BDA p.407]．そのこともあり，プロビットモデルのベイズ推論は計量経済学や政治科学で広く使われている手法となったが，ロジットモデルのベイズ推論の応用は遅れていた [@Polson+2013]．

しかし実はロジットモデルの事後分布 $\pi$ も正規分布の Pólya-Gamma 混合として表すことができ，データ拡張によって効率的な Gibbs サンプラーを構成することができる [@Polson+2013]．現在ではこのデータ拡張 Gibbs サンプラーが，標準的な事後分布サンプラーとなっている．

### Gibbs サンプラーの課題：不均衡データ

データもモデルも大規模になっていく現代では，このようなデータ拡張に基づく Gibbs サンプラーは特定の条件が揃うと極めて収束が遅くなる場面が少なくないことが明らかになってきている．

そのうちの１つのパターンが大規模な **不均衡データ** [@Johndrow+2019]，すなわち，特定のラベルが極めて稀少なカテゴリカルデータである．

このようなデータに対しては，プロビットモデルやロジットモデルに限らず，ほとんど全てのデータ拡張に基づく Gibbs サンプラーが低速化することが報告されている：

> We have found that this behavior occurs routinely, essentially regardless of the type and complexity of the statistical model, if the data are large and imbalanced. [@Johndrow+2019 p.1395]

### 実験：不均衡データでの収束鈍化

$p=1$ 次元のプロビットモデル
$$
\sum_{i=1}^n y_i\,\bigg|\,(n,\theta)\sim\Bin(n,F(\theta)),\qquad\theta\sim\rN_1(0,B),B=100,
$$
しかも説明変数なしの切片項のみの状況を考える：
$$
x_i\equiv1,\qquad p_0(\xi)d\xi=\rN(a,B),\qquad a=0,
$$

この場合，ポテンシャルは次のように表される：
$$
-U(\xi)=\xi\sum_{i=1}^ny_i-n\log(1+e^{\xi})-\frac{(\xi-a)^2}{2B}-\frac{1}{2}\log2\pi B.
$$

ここまで単純化した設定でも，観測 $y_i$ のカテゴリが不均衡ならば，前述の Gibbs サンプラーの収束鈍化が見られることを検証する：
$$
\sum_{i=1}^ny_i=1.
$$

::: {.callout-important title="展開してコードを見る" collapse="true" icon="false"}

Metropolis-Hastings 法は，Turing Institute による Julia の `AdvancedMH.jl` パッケージなどを通じて実装することができる：

::: {#lst-listing3}
:::


In [ ]:
using AdvancedMH
using Distributions
using MCMCChains
using ForwardDiff
using StructArrays
using LinearAlgebra
using LogDensityProblems
using LogDensityProblemsAD

(a,B) = (0,100.0)

# Define the components of a basic model.
struct LogTargetDensity_Logistic
    a::Float64
    B::Float64
    n::Int64
end

LogDensityProblems.logdensity(p::LogTargetDensity_Logistic, ξ) = -log(2π * p.B) - (ξ[1] - p.a)^2/(2 * p.B) + ξ[1] - p.n * log(1 + exp(ξ[1]))
LogDensityProblems.dimension(p::LogTargetDensity_Logistic) = 1
LogDensityProblems.capabilities(::Type{LogTargetDensity_Logistic}) = LogDensityProblems.LogDensityOrder{0}()

function MHSampler(n::Int64; discard_initial=30000)

    model_with_ad = LogDensityProblemsAD.ADgradient(Val(:ForwardDiff), LogTargetDensity_Logistic(a, B, n))

    spl = RWMH(MvNormal(zeros(1), I))

    chain = sample(model_with_ad, spl, 50000; chain_type=Chains, param_names=["ξ"])

    return chain
end

# ξ_vector = MHSampler(10000)
# plot(ξ_vector, title="Plot of \$\\xi\$ values", xlabel="Index", ylabel="ξ", legend=false, color="#78C2AD")

In [ ]:
using DataFrames
using Plots

n_list = [10, 100, 1000, 10000]

elapsed_time_Metropolis = @elapsed begin
    chains = [MHSampler(n) for n in n_list]
end

autos = [DataFrame(autocor(chain, lags=1:100)) for chain in chains]

MHChain = chains

combined_df = vcat(autos..., source=:chain)

lag_columns = names(combined_df)[2:101]
lags = 1:100

p_Metropolis = plot(
    title = "Metropolis",
    xlabel = "Lag",
    ylabel = "Autocorrelation",
    legend = :topright,
    #background_color = "#F0F1EB"
)

for (i, n) in zip(1:4, n_list)
    plot!(
        p_Metropolis,
        lags,
        Array(combined_df[i, lag_columns]),
        label = "n = $n",
        linewidth = 2
    )
end

パッケージ `PolyaGammaSamplers` は現在，過去のバージョンの依存関係を必要とするので，グローバルの環境とは分離しておくのが良い．

ここでは，Pólya-Gamma 分布のサンプラーの実装 [`PolyaGammaSamplers`](https://github.com/igutierrezm/PolyaGammaSamplers.jl/blob/master/src/polyagammapswsampler.jl) を参考にして，直接次のように定義する．


In [ ]:
using Random
using StatsFuns

struct PolyaGammaPSWSampler{T <: Real} <: Sampleable{Univariate, Continuous}
    b::Int
    z::T
end

struct JStarPSWSampler{T <: Real} <: Sampleable{Univariate, Continuous}
    z::T
end

function Base.rand(rng::AbstractRNG, s::PolyaGammaPSWSampler)
    out = 0.0
    s_aux = JStarPSWSampler(s.z / 2)
    for _ in 1:s.b
        out += rand(rng, s_aux) / 4
    end
    return out
end

function Base.rand(rng::AbstractRNG, s::JStarPSWSampler)
    z = abs(s.z)  # modified to avoid negative z
    t = 0.64
    μ = 1 / z
    k = π^2 / 8 + z^2 / 2
    p = (π / 2 / k) * exp(- k * t) 
    q = 2 * exp( - z) * cdf(InverseGaussian(μ, 1.0), t)
    while true
        # Simulate a candidate x
        u = rand(rng)
        v = rand(rng)
        if (u < p / (p + q))
            # (Truncated Exponential)
            e = randexp(rng)
            x = t + e / k
        else
            # (Truncated Inverse Gaussian)
            x = randtigauss(rng, z, t)
        end
        # Evaluate if the candidate should be accepted
        s = a_xnt(x, 0, t)
        y = v * s
        n = 0
        while true
            n += 1
            if (n % 2 == 1)
                s += a_xnt(x, n, t)
                y > s && break
            else
                s -= a_xnt(x, n, t)
                y < s && return x
            end
        end
    end
end

# Return ``a_n(x)`` for a given t, see [1], eqs. (12)-(13)
# Equations (12)-(13) in [1]
# Note: 
# This is a literal transcription from the article's formula
# except for the letter case
function a_xnt(x::Real, n::Int, t::Real)
    x ≤ t ? a_xnt_left(x, n, t) : a_xnt_right(x, n, t)
end

# Return ``a_n(x)^L`` for a given t
# Equation (12) in [1]
# Note: 
# This is a literal transcription from the article's formula
# except for the letter case
function a_xnt_left(x::Real, n::Int, t::Real)
    π * (n + 0.5) * (2 / π / x)^(3 / 2) * exp(- 2 * (n + 0.5)^2 / x)
end

# Return ``a_n(x)^R`` for a given t, see [1], eq. (13)
# Equation (13) in [1]
# Note: 
# This is a literal transcription from the article's formula
# except for the letter case
function a_xnt_right(x::Real, n::Int, t::Real)
    π * (n + 0.5) * exp(- (n + 0.5)^2 * π^2 * x / 2)
end

# Simulate from an IG(μ, 1) distribution
# Algorithms 2-3 in [1]'s supplementary material
# Note: 
# This is a literal transcription from the article's pseudo code
# except for the letter case
function randtigauss(rng::AbstractRNG, z::Real, t::Real)
    1 / z > t ? randtigauss_v1(rng, z, t) : randtigauss_v2(rng, z, t)
end

# Simulate from an IG(μ, 1) distribution, for μ := 1 / z > t;
# Algorithms 2 in [1]'s supplementary material
# Note:
# This is a literal transcription from the article's pseudo code
# except for the letter case and one little a detail: the 
# original condition  `x > R` must be replaced by `x > t`
function randtigauss_v1(rng::AbstractRNG, z::Real, t::Real)
    x = t + one(t)
    α = zero(t)
    while rand(rng) > α
        e = randexp(rng) # In [1]: E 
        é = randexp(rng) # In [1]: E'
        while e^2 > (2 * é / t)
            e = randexp(rng)
            é = randexp(rng)
        end
        x = t / (1 + t * e)^2 
        α = exp(- z^2 * x / 2)
    end
    return x
end

# Simulate from an IG(μ, 1) distribution, for μ := 1 / z ≤ t
# Algorithms 3 in [1]'s supplementary material
# Note: This is a literal transcription from the article's pseudo code
function randtigauss_v2(rng::AbstractRNG, z::Real, t::Real)
    x = t + one(t)
    μ = 1 / z
    while x > t 
        y = randn(rng)^2
        x = μ + μ^2 * y / 2 - μ * √(4 * μ * y + (μ * y)^2) / 2
        if rand(rng) > μ / (μ + x)
            x = μ^2 / x
        end
    end
    return x
end

In [ ]:
# using PolyaGammaSamplers

function PGSampler(n::Int64; discard_initial=30000, iter_number=50000, initial_ξ=0.0, B=100)

    λ = 1 - n/2

    ξ_list = [initial_ξ]
    ω_list = []

    while length(ξ_list) < iter_number
        ξ = ξ_list[end]
        ω_sampler = PolyaGammaPSWSampler(n, ξ)
        ω_new = rand(ω_sampler)
        push!(ω_list, ω_new)
        ξ_sampler = Normal((ω_new + B^(-1))^(-1) * λ, (ω_new + B^(-1))^(-1))
        ξ_new = rand(ξ_sampler)
        push!(ξ_list, ξ_new)
    end

    return Chains(ξ_list[discard_initial+1:end])
end

function Distributions.mean(s::PolyaGammaPSWSampler)
    s.b * inv(2.0 * s.z) * tanh(s.z / 2.0)
end

function Distributions.var(s::PolyaGammaPSWSampler)
    s.b * inv(4 * s.z^3) * (sinh(s.z) - s.z) * (sech(s.z / 2)^2)
end

In [ ]:
elapsed_time_PolyaGamma = @elapsed begin
    chains = [PGSampler(n) for n in n_list]
end
autos = [DataFrame(autocor(chain, lags=1:100)) for chain in chains]

PGChain = chains

combined_df = vcat(autos..., source=:chain)

lag_columns = names(combined_df)[2:101]
lags = 1:100

p_PolyaGamma = plot(
    title = "Pólya-Gamma",
    xlabel = "Lag",
    ylabel = "Autocorrelation",
    legend = (0.65, 0.35),
    #background_color = "#F0F1EB"
)

for (i, n) in zip(1:4, n_list)
    plot!(
        p_PolyaGamma,
        lags,
        Array(combined_df[i, lag_columns]),
        label = "n = $n",
        linewidth = 2,
    )
end

In [ ]:
println("Elapsed time: $elapsed_time_Metropolis seconds v.s. $elapsed_time_PolyaGamma seconds")

PG サンプラーは MH 法に比べ恐ろしいほどに時間がかかる．これは，`Turing` のパッケージの最適化が優秀であるのか，Pólya-Gamma サンプラーの宿命であるのか，引き続き調べる必要がある．

:::


In [ ]:
#| code-fold: true
#| echo: false
plot(p_Metropolis, p_PolyaGamma, layout=(1,2), #background_color = "#F0F1EB"
)
#savefig("Logistic_WhiteBackground.svg")

### 理論：収束鈍化の理由

前節の $p=1$ の不均衡プロビットモデルの下での計算複雑性のオーダーは，

* Metropolis 法では最悪で $(\log n)^3$
* Gibbs サンプラーでは最高でも $n^{3/2}(\log n)^{2.5}$

になることが示されている [Section 4 @Johndrow+2019]．

その理由は，提案分布と対象分布のズレに由来することも [@Johndrow+2019] は明らかにしている．

$\sum_{i=1}^ny_i$ の値を固定して $n\to\infty$ の極限を取った場合，事後分布は次のように負方向にスライドしながら，幅が狭まっていく．

その幅の縮小レートは $n^{-1/2}$ ではなく，約 $(\log n)^{-1}$ になる．


In [ ]:
#| echo: false
using StatsPlots
using LaTeXStrings

plot(
    plot(MHChain[1], title=L"n=10", color="#78C2AD"),
    plot(MHChain[2], title=L"n=100", color="#78C2AD"),
    plot(MHChain[3], title=L"n=1000", color="#78C2AD"),
    plot(MHChain[4], title=L"n=10000", color="#78C2AD"),
    layout=(4,1),
    size=(1000, 800),
    #background_color = "#F0F1EB"
)

一方で提案分布は $\xi_t$ をモードとした場合，$\xi_{t+1}$ もモードの周りに幅 $\frac{(\log n)^{3/2}}{n^{1/2}}$ で集中してしまう．すなわち，提案のステップサイズが事後分布のスケールに比べて極めて小さくなってしまう．


In [ ]:
#| echo: false
plot(
    plot(PGChain[1], title=L"n=10", color="#78C2AD"),
    plot(PGChain[2], title=L"n=100", color="#78C2AD"),
    plot(PGChain[3], title=L"n=1000", color="#78C2AD"),
    plot(PGChain[4], title=L"n=10000", color="#78C2AD"),
    layout=(4,1),
    size=(1000, 800),
    #background_color = "#F0F1EB"
)

MH 法でサンプルした事後分布に比べて，より鋭くなっていることがわかるだろう（$y$ 軸のスケールに注目）．

Gibbs サンプラーではステップサイズがあまりに小さくなってしまい，分布の十分な探索が阻害され，サンプル間の自己相関が高くなってしまうという問題が起こるようである．

## Zig-Zag サンプラーによる解決

後編に続く：

::: {#lst-listing}
:::

## 文献紹介 {.appendix}

ロジスティック回帰やプロビット回帰などの２項回帰モデルは，広くベイズ計算手法のベンチマークとして用いられるモデルである [@Chopin-Ridgway2017]．

二項回帰モデルにおいて観測 $y_{i}$ の値が激しく偏っていた場合，Gibbs サンプラーの収束に問題が起きることは [@Johndrow+2019] が指摘している．

そもそも中規模のデータセットでは Gibbs サンプラーは乱歩 MH などの汎用手法と比べてほとんど同等の性能であることが [@Chopin-Ridgway2017] の数値実験で確認されている．

この問題が Zig-Zag サンプラーを用いれば，重点荷重＋ミニバッチサブサンプリングのアイデアで比較的簡単に解決できることが [@Sen+2020] で実証された．